In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

class MyTreeClf():
    def __init__(self, max_depth=5, min_samples_split=2, max_leafs=20, bins = None, criterion = 'entropy'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.leafs_cnt = 1
        self.tree = {}
        self.bins = bins
        self.feature_bins = {}
        self.criterion = criterion
        self.fi = {}
        self.N=None

        if self.bins is not None:
            for col in X.columns:
                unique_values = sorted(X[col].unique())
                if len(unique_values) - 1 <= self.bins:
                    self.feature_bins[col] = [(unique_values[i] + unique_values[i+1]) / 2 for i in range(len(unique_values) - 1)]
                else:
                    hist, bin_edges = np.histogram(X[col], bins=self.bins)
                    self.feature_bins[col] = bin_edges[:-1]

    def __str__(self):
        return "MyTreeClf class: " + ", ".join(('{}={}'.format(item, self.__dict__[item]) for item in self.__dict__))

    def __repr__(self):
        return "MyTreeClf class: " + ", ".join(('{}={}'.format(item, self.__dict__[item]) for item in self.__dict__))

    def get_best_split(self, X, y):
        best_score = -1
        best_col_name = None
        best_split_value = None

        for col_name in X.columns:
            split_values = self.feature_bins.get(col_name, [])
            if not np.array(split_values).any():
                unique_values = sorted(X[col_name].unique())
                split_values = [(unique_values[i] + unique_values[i+1]) / 2 for i in range(len(unique_values) - 1)]

            for split_value in split_values:
                left_indices = X[col_name] <= split_value
                right_indices = X[col_name] > split_value

                if self.criterion == 'entropy':
                    left_entropy = self._entropy(y[left_indices])
                    right_entropy = self._entropy(y[right_indices])
                    weighted_entropy = (len(y[left_indices]) / len(y)) * left_entropy + (len(y[right_indices]) / len(y)) * right_entropy
                    score = self._entropy(y) - weighted_entropy
                elif self.criterion == 'gini':
                    left_gini = self._gini(y[left_indices])
                    right_gini = self._gini(y[right_indices])
                    weighted_gini = (len(y[left_indices]) / len(y)) * left_gini + (len(y[right_indices]) / len(y)) * right_gini
                    score = self._gini(y) - weighted_gini
                else:
                    raise ValueError("Invalid criterion. Choose 'entropy' or 'gini'.")

                if score > best_score:
                    best_score = score
                    best_col_name = col_name
                    best_split_value = split_value

        return best_col_name, best_split_value, best_score

    def _entropy(self, y):
          _, counts = np.unique(y, return_counts=True)
          probabilities = counts / len(y)
          entropy = -np.sum(probabilities * np.log2(probabilities, where=(probabilities != 0)))
          return entropy

    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities ** 2)
        return gini


    def fit(self, X, y):
        self.N=len(y)
        self.leafs_cnt = 1
        self.fi = {col: 0 for col in X.columns}
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        proportion_ones = len(y[y == 1]) / len(y) if len(y) else 0
        if proportion_ones == 1 or proportion_ones == 0 or depth >= self.max_depth or len(y) < self.min_samples_split or (self.leafs_cnt > 1 and self.leafs_cnt >= self.max_leafs):
            return {'leaf': True, 'value': proportion_ones}

        best_col, best_split, ig = self.get_best_split(X, y)
        left_indices = X[best_col] <= best_split
        right_indices = X[best_col] > best_split
        self.leafs_cnt += 1

        N = self.N
        Np = len(y)
        Nl = len(y[left_indices])
        Nr = len(y[right_indices])
        I = self._entropy(y) if self.criterion == 'entropy' else self._gini(y)
        Il = self._entropy(y[left_indices]) if self.criterion == 'entropy' else self._gini(y[left_indices])
        Ir = self._entropy(y[right_indices]) if self.criterion == 'entropy' else self._gini(y[right_indices])


        self.fi[best_col] += (Np / N) * (I - (Nl / Np) * Il - (Nr / Np) * Ir)

        return {
            'col': best_col,
            'split': best_split,
            'left': self._build_tree(X[left_indices], y[left_indices], depth + 1),
            'right': self._build_tree(X[right_indices], y[right_indices], depth + 1)
        }

    def predict_proba(self, X):
        return [self._predict_proba_single(row) for _, row in X.iterrows()]

    def _predict_proba_single(self, row):
        node = self.tree
        while 'leaf' not in list(node.keys()):
            if row[node['col']] <= node['split']:
                node = node['left']
            else:
                node = node['right']
        return node['value']

    def predict(self, X):
        return [1 if p > 0.5 else 0 for p in self.predict_proba(X)]


    def print_tree(self, node=None, depth=0):
        if node is None:
            node = self.tree

        if 'leaf' in node:
            print(f"{' ' * depth}Leaf: {node['value']}")
        else:
            print(f"{' ' * depth}{node['col']} > {node['split']}")
            self.print_tree(node['left'], depth + 1)
            self.print_tree(node['right'], depth + 1)

    def get_feature_importances(self):
        return self.fi







# Sample data
X, y = make_classification(n_samples=150, n_features=5, n_informative=3, random_state=42)
X = pd.DataFrame(X).round(2)
y = pd.Series(y)
X.columns = [f'col_{col}' for col in X.columns]
test = X.sample(20, random_state=42)

tree = MyTreeClf(
    max_depth = 3,
    min_samples_split = 2,
    max_leafs = 1
)
tree.fit(X, y)

tree.print_tree()

print(np.sum(tree.predict_proba(X)))

col_0 > -0.1
 Leaf: 0.8235294117647058
 Leaf: 0.07692307692307693
75.0
